In [1]:
from DETR_model import DinoDetrMaskMul
from torch.utils.data import DataLoader
import torch
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
from polygon import RasLoss, SoftPolygon
from Loader_17 import DAVIS_Rawset, DAVIS_Infer, DAVIS_Dataset
import logging
import json

In [2]:
model_name = "DETR_davis_mb"
log_path = f"./log/{model_name}.log"
logging.basicConfig(
    filename=log_path,
    level=logging.INFO,
    format="%(asctime)s %(levelname)-8s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logging.info(f"Start training {model_name}")

In [8]:
# load the dataset
train_rawset = DAVIS_Rawset(is_train=True)

train_rawset.data_set = train_rawset.data_set[:10]

train_dataset = DAVIS_Dataset(train_rawset)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
train_infer = DAVIS_Infer(train_rawset)

val_rawset = DAVIS_Rawset(is_train=False)

val_rawset.data_set = val_rawset.data_set[:10]

val_infer = DAVIS_Infer(val_rawset)

In [9]:
# load the model
model = DinoDetrMaskMul(mul_before=True).cuda()

# load the loss function
ras_loss = RasLoss().cuda()

# load the optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)

Using cache found in /zhome/32/f/202284/.cache/torch/hub/mhamilton723_FeatUp_main
Using cache found in /zhome/32/f/202284/.cache/torch/hub/facebookresearch_dino_main
/work3/s232248/miniconda3/envs/ras/lib/python3.12/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [10]:
train_loss_dict = {}
train_iou_dict = {}
val_iou_dict = {}
best_val_iou = 0
interval_epochs = 15
interval_steps = 40
epoch_index = 0

In [11]:
for interval in range(interval_steps):
    for e in range(interval_epochs):
        model.train()
        mean_loss = 0
        for (
            video_idx,
            frame_idx,
            fir_img,
            fir_sgm,
            pre_img,
            pre_sgm,
            cur_img,
            cur_sgm,
        ) in train_loader:
            optimizer.zero_grad()
            fir_img = fir_img.cuda()
            fir_sgm = fir_sgm.cuda()
            pre_img = pre_img.cuda()
            pre_sgm = pre_sgm.cuda()
            cur_img = cur_img.cuda()
            cur_sgm = cur_sgm.cuda()
            pred_bou = model(fir_img, fir_sgm, pre_img, pre_sgm, cur_img)
            loss = ras_loss(pred_bou, cur_sgm)
            loss.backward()
            optimizer.step()
            mean_loss += loss.item()
        mean_loss /= len(train_loader)
        train_loss_dict[epoch_index] = mean_loss
        with open(f"./log/{model_name}_loss.json", "w") as f:
            json.dump(train_loss_dict, f)
        # save the model
        torch.save(
            model.state_dict(),
            f"./model/{model_name}.pth",
        )
        logging.info(f"Epoch {epoch_index}, Loss: {mean_loss}")
        epoch_index += 1
    train_infer.infer_model(model)
    train_iou = train_infer.get_total_iou()
    train_iou_dict[epoch_index] = train_iou
    val_infer.infer_model(model)
    val_iou = val_infer.get_total_iou()
    val_iou_dict[epoch_index] = val_iou
    if val_iou > best_val_iou:
        best_val_iou = val_iou
        torch.save(
            model.state_dict(),
            f"./model/{model_name}_best.pth",
        )
    logging.info(f"Epoch {epoch_index}, Train IoU: {train_iou}, Val IoU: {val_iou}")
    with open(f"./log/{model_name}_train_iou.json", "w") as f:
        json.dump(train_iou_dict, f)
    with open(f"./log/{model_name}_val_iou.json", "w") as f:
        json.dump(val_iou_dict, f)
            

KeyboardInterrupt: 